# EDA for Flavor Network Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# This is our regional recipe component data
dishes = pd.read_csv('Data/dish_components.csv',error_bad_lines=False)

# Transpose makes dishes easier to index and we drop a few empty rows
dishes = dishes.T
dishes = dishes.dropna(how='all')

# This is our shared compound data
compounds = pd.read_csv('Data/shared_compounds.csv')
pd.DataFrame(compounds)



/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
# This is the GroupBy object that allows lookup of number of shared compounds
combinator = compounds.copy()
combinatrix = combinator.groupby(['First Ingredient','Second Ingredient']).sum()
combinatrix

# of Shared Compounds
First Ingredient     Second Ingredient                                
abies_alba           abies_canadensis                                1
                     abies_concolor                                  1
                     abies_sibirica                                  1
                     callitris_drummondii                            1
                     callitris_glauca                                1
...                                                                ...
yellow_mombin        ylangylang                                      1
yellow_passion_fruit yellow_passion_fruit_juice                      9
ylangylang           ylangylang_oil                                  5
                     yogurt                                          1
zanthoxylum_rhetsa   zanthoxylum_rhetsa_oil                          1

[221777 rows x 1 columns]

# Make a zeros matrix with correct labels

In [9]:
# Make a list of all unique ingredients.
Ingredients = set(compounds['First Ingredient'])
len(Ingredients)
Ingredients2 = set(compounds['Second Ingredient'])
Ingredients.update(Ingredients2)
len(Ingredients)
unique = sorted(list(Ingredients))
dimen = len(unique)

In [16]:
# Zeros matrix with columns and index of unique ingredients
flavor_mat = np.ma.zeros((dimen,dimen), dtype='Int16')
flavor_matrix = pd.DataFrame(flavor_mat, index=unique, columns=unique)
flavor_matrix.head(25)

# Make 2 matrices: shared compounds, and ingredients in the same recipe

## Shared compounds matrix:

In [19]:
# List shared flavor compounds by ingredient. Remove 'int' to double-check locations.
long_list = []

for i in range(0,len(unique)):
    for j in range(0,len(unique)):
        try: 
            long_list.append(int(combinatrix.loc[unique[i], unique[j]]))
        except KeyError:
            long_list.append(0)

KeyboardInterrupt: 

In [111]:
# Make flavor compounds list an array
long_list_array = np.array(long_list).reshape((len(unique),len(unique)))

# Put array into dataframe with column and index labels
comp_share = pd.DataFrame(long_list_array, index=unique, columns=unique)

# Make the matrix symmetrical here by adding the transposed matrix
comp_share = comp_share + comp_share.T

In [112]:
# Save to csv
#comp_share.to_csv('Shared_Compounds_Dense.csv')

In [21]:
comp_share = pd.read_csv('Shared_Compounds_Dense.csv')

## Concurrent ingredients from recipes matrix

In [22]:
# Define function for getting a list of components in recipes. Recipes assumed to be in columns here.
def get_rec_compon(recipes):
    rec_compon = []
    for i in range(0,len(recipes.columns)):
        irec = recipes[i]
        irec = irec.dropna()
        irec = irec[1:][0:]
        icomp = [component for component in irec]
        rec_compon.append(icomp)
    return rec_compon        

In [23]:
# This is a list of components
dish_compon = get_rec_compon(dishes)

In [145]:
# Copy the zeros matrix to make sure we're starting from scratch
rec_compon_matrix = flavor_matrix.copy()

# Now we can loop through our list of dish components.
for components in dish_compon:
    for i in range(0,len(components)):
        for j in range(0,len(components)):
            if components[i] != components[j]:
                try:
                    rec_compon_matrix.loc[components[i]][components[j]] += 1
                except KeyError:
                    #print("Couldn't find ", str(components[i]), "/", str(components[j]))
                    continue
            else:
                continue

In [146]:
# Note on above cell: 'starch' occurs in many recipes, but shares no flavor compounds with anything else
# therefore it is not in the list of unique ingredients. There are probably others like this.

In [147]:
# Save recipe components matrix
#rec_compon_matrix.to_csv("Recipe_Components_Dense.csv")

In [24]:
rec_compon_matrix = pd.read_csv('Recipe_Components_Dense.csv')

# Drill down into specific recipes and look at the flavor compound relationships, then visualize with a Chort chart

In [116]:
# Take a column from dishes, drop nan and region of origin
recipe = dishes[1104]
recipe = recipe.dropna()
recipe = recipe[1:][0:]
recipe

Ingredient 1     vinegar
Ingredient 2    cucumber
Ingredient 3     seaweed
Ingredient 4      radish
Ingredient 5        rice
Ingredient 6      wasabi
Name: 1104, dtype: object

In [117]:
# List comprehension for recipe
compon = [ing for ing in recipe]

In [78]:
# Testing loc on GroupBy object
combinatrix.loc[compon[3],compon[1]]

Shared    2
Name: (garlic, ginger), dtype: int64

In [61]:
# List shared flavor compounds in recipe. Remove 'int' to check locations.
components = []

for i in range(0,len(compon)):
    for j in range(0,len(compon)):
        try: 
            components.append(int(combinatrix.loc[compon[i], compon[j]]))
        except KeyError:
            components.append(0)

In [62]:
# Make list into array
dish_shared = np.array(components).reshape((len(recipe),len(recipe)))

In [63]:
# Creates DataFrame and Matrix as a list.
cereals = pd.DataFrame(dish_shared, index=compon, columns=compon)

# Make the matrix symmetrical here by adding the transposed matrix
cereals = cereals + cereals.T

hops = cereals.values.tolist()

In [64]:
# Values for Chord chart
hops

[[0, 28, 0, 7, 0],
 [28, 0, 0, 10, 0],
 [0, 0, 0, 0, 0],
 [7, 10, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [65]:
# Labels for Chord chart
compon

['eel', 'rice', 'seaweed', 'vinegar', 'wasabi']

In [119]:
from chord import Chord

In [128]:
#First Chord chart
#Chord(hops, compon).to_html('chord.html')